In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torch.nn as nn
import copy
import time
import os
import argparse
from torch.utils.data import Subset
import csv
device = torch.device('cuda')
from ResNet import *
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,precision_score,recall_score
from sklearn.metrics import ConfusionMatrixDisplay

In [3]:
loaded_data1 = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/data.pt")
images= loaded_data1['images']
# images.dtype
# labels = loaded_data1['labels']
images = images / 255
images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
print(images.dtype)
labels = loaded_data1['labels']


torch.float32


In [4]:

loaded_data = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/test_data.pt")
images1 = loaded_data['images']
labels1 = loaded_data['labels']
images1 = images1 / 255
images1 = (images1 - images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)
# print(images1.shape)
# print(labels1.shape)
# print(images.shape)
# print(labels.shape)


In [5]:
train_images = images1
train_labels = labels1
test_images = images
test_labels = labels

In [6]:
print(train_labels.dtype)
print(test_labels.dtype)

torch.uint8
torch.uint8


In [7]:
s=0.5
color_jitter = transforms.ColorJitter(0.4*4, 0.4*s, 0.4*s, 0.2*s)

transform_eval = transforms.Compose([
        # transforms.RandomResizedCrop((224,224), scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([color_jitter], p=0.5), 
        transforms.RandomGrayscale(p=0.4),
        # transforms.Resize(224),
        # transforms.ToTensor(),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

In [8]:
labels1.max()
labels.max()

tensor(1, dtype=torch.uint8)

In [9]:
train_images = transform_eval(train_images)

In [10]:
test_images.dtype

torch.float32

In [11]:
train_labels_count = (train_labels == 1).sum()
test_labels_count = (test_labels == 1).sum()

print("Number of train labels with value 1:", train_labels_count)
print("Number of test labels with value 1:", test_labels_count)

Number of train labels with value 1: tensor(1042)
Number of test labels with value 1: tensor(1697)


In [12]:
train_set=torch.utils.data.TensorDataset(train_images,train_labels)
test_set=torch.utils.data.TensorDataset(test_images,test_labels)
train_loader=torch.utils.data.DataLoader(train_set,batch_size=512,shuffle=True,num_workers=8)
test_loader=torch.utils.data.DataLoader(test_set,batch_size=512,shuffle=False,num_workers=8)

In [13]:
base_encoder = torchvision.models.efficientnet_b0(pretrained=True)
n_features = base_encoder.classifier[1].in_features
base_encoder.classifier = nn.Sequential(
    nn.Linear(n_features, n_features),
    nn.ReLU()
)
base_encoder.linear = nn.Sequential(
    nn.Linear(n_features, 2))

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
classes = ('yes', 'no')
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
model = base_encoder
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


In [15]:

def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f'{epoch} Loss: {train_loss:.4f} Acc: {correct/total:.4f}')

best_acc=0
accuracies=[]

def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    acc = 0
    all_targets = []
    all_preds = []

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            all_targets.extend(targets.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    acc = 100. * correct / total
    print(f'{epoch} Test Loss: {test_loss / (batch_idx + 1):.4f} Test Acc: {correct / total:.4f}')
    print(f'Test Accuracy: {acc:.2f}%')
    accuracies.append(acc)

    return all_preds, all_targets

start_epoch = 0 
for epoch in range(start_epoch, start_epoch+100):
    train(epoch)
    y_pred, y_true = test(epoch)



Epoch: 0
0 Loss: 130.4335 Acc: 0.3392
0 Test Loss: 6.6654 Test Acc: 0.2855
Test Accuracy: 28.55%

Epoch: 1
1 Loss: 91.8643 Acc: 0.5569
1 Test Loss: 5.1675 Test Acc: 0.5568
Test Accuracy: 55.68%

Epoch: 2
2 Loss: 58.8973 Acc: 0.6649
2 Test Loss: 2.1189 Test Acc: 0.8707
Test Accuracy: 87.07%

Epoch: 3
3 Loss: 29.5318 Acc: 0.8671
3 Test Loss: 3.5378 Test Acc: 0.7495
Test Accuracy: 74.95%

Epoch: 4
4 Loss: 9.2672 Acc: 0.9628
4 Test Loss: 4.9654 Test Acc: 0.6517
Test Accuracy: 65.17%

Epoch: 5
5 Loss: 4.1199 Acc: 0.9732
5 Test Loss: 4.7904 Test Acc: 0.6631
Test Accuracy: 66.31%

Epoch: 6
6 Loss: 2.7159 Acc: 0.9821
6 Test Loss: 5.6943 Test Acc: 0.6763
Test Accuracy: 67.63%

Epoch: 7
7 Loss: 1.9901 Acc: 0.9855
7 Test Loss: 5.9630 Test Acc: 0.6828
Test Accuracy: 68.28%

Epoch: 8
8 Loss: 1.4447 Acc: 0.9896
8 Test Loss: 6.5499 Test Acc: 0.6804
Test Accuracy: 68.04%

Epoch: 9
9 Loss: 1.2921 Acc: 0.9899
9 Test Loss: 6.6579 Test Acc: 0.6816
Test Accuracy: 68.16%

Epoch: 10
10 Loss: 1.1585 Acc: 0.9

In [26]:
torch.save(model.state_dict(), '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/delhi_whole_data_train_model.pt')


In [27]:
y_true = y_true.clone().detach()
y_pred = y_pred.clone().detach()

: 

In [24]:
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score
accuracy = accuracy_score( y_pred,y_true)
precision = precision_score(y_pred,y_true)
recall = recall_score(y_pred,y_true)
f1 = f1_score(y_pred,y_true)

In [25]:
print("Accuracy:",accuracy)
print("Precision:",precision)
print("Recall:",recall)
print("F1:",f1)

Accuracy: tensor(0.6745)
Precision: tensor(0.0409)
Recall: tensor(0.1344)
F1: tensor(0.0627)
